In [2]:
import requests	
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from sqlalchemy import create_engine
import config
from unidecode import unidecode
import datetime

In [6]:
fs_urls = pd.read_csv('data/usfs_sites.csv')

In [7]:
fs_urls

,facilityname,url
0,whispering falls campground,Willamette National Forest - Whispering Falls ...
1,lost lake campground,Mt. Hood National Forest - Lost Lake Campgroun...
2,lake harriet campground,Mt. Hood National Forest - Lake Harriet Campgr...
3,kinnikinnick (laurance lake) campground,Mt. Hood National Forest - Kinnikinnick (Laura...
4,fifteenmile campground,Mt. Hood National Forest - Fifteenmile Campgro...
5,clear lake campground,Mt. Hood National Forest - Clear Lake Campgrou...
6,camp creek campground,Mt. Hood National Forest - Camp Creek Campgrou...
7,badger lake campground,Mt. Hood National Forest - Badger Lake Campgro...
8,eagle creek campground,Columbia River Gorge National Scenic Area - Ea...


In [59]:
site_url = "http://" + config.LAMP_IP + "/" + fs_urls.iloc[1]['url']
cg_name = fs_urls.iloc[1]['facilityname']

In [36]:
cg_req = requests.get(site_url)
cg_soup = BeautifulSoup(cg_req.text, 'lxml')

Extracting Area Status information

In [33]:
try :
    for strong_tag in cg_soup.find_all('strong'):
        if ('Area Status' in unidecode(strong_tag.text)):
            status = unidecode(strong_tag.next_sibling).strip()
except Exception:
    print('couldnt get area status')

In [34]:
status

'Open'

Extracting lat/long information

In [41]:
try :
    lat = cg_soup.find_all('div', text=re.compile('Latitude'))
    div = [row.next_sibling.next_sibling for row in lat]
    latitude  = div[0].text.strip()

except Exception:
    print('couldnt get location info')

In [43]:
latitude

'45.50080'

In [47]:
def get_location(soup, search_text):
    try :
        divs = soup.find_all('div', text=re.compile(search_text))
        loc_div = [row.next_sibling.next_sibling for row in divs]
        return loc_div[0].text.strip()
    except Exception as ex:
        print("get_location: couldnt extract " + search_text)
        return ""

In [50]:
cg_lat = get_location(cg_soup, 'Latitude')
cg_long = get_location(cg_soup, 'Longitude')
cg_elev = get_location(cg_soup, 'Elevation')

In [52]:
print(str(cg_lat) + "," + str(cg_long) + " elevation:" + str(cg_elev))

45.50080,-121.81641 elevation:3200


Extracting campground info

In [53]:
try :
    tables = cg_soup.find_all('div', {'class': 'tablecolor'})
except Exception:
    print('couldnt get tables')

In [60]:
try :
    print(len(tables))
    rows = tables[0].find_all('tr')
    for row in rows:
        if row.th.text == 'Reservations:':
            reservations = unidecode(row.td.text).strip()
        if row.th.text == 'Open Season:':
            openseason = unidecode(row.td.text).strip()
        if row.th.text == 'Current Conditions:':
            conditions = unidecode(row.td.text).strip()
        if row.th.text == 'Water:':
            water = unidecode(row.td.text).strip()
        if row.th.text == 'Restroom:':
            restroom = unidecode(row.td.text).strip()
except Exception as ex:
    print('couldnt get basic campground info')
    print(ex)
    
df_info = pd.DataFrame({
        'FacilityName':[cg_name],
        'Reservations':[reservations],
        'OpenSeason':[openseason],
        'CurrentConditions':[conditions],
        'Water':[water],
        'Restroom':[restroom]
    })

10


In [61]:
df_info

,CurrentConditions,FacilityName,OpenSeason,Reservations,Restroom,Water
0,OPEN,lost lake campground,April,This site can be reserved by calling Toll Free...,Vault Toilet (18),Potable Water


In [3]:
def extract_cg_info(campgrounds) :
    df_cg = pd.DataFrame()
    for index, campground in campgrounds.iterrows():
        latitude = longitude = elevation = site_id = numsites = conditions = ""
        fees = openseason = usage = water = restroom = status = ""
        site_url = "http://" + config.LAMP_IP + "/" + campground['url']
        print(campground['facilityname'] + '\t' + site_url)
        try :
            cg_req = requests.get(site_url)
            cg_soup = BeautifulSoup(cg_req.text, 'lxml')
        except Exception as ex :
            print('couldnt get site_url ' + site_url)
            print(ex)
            continue

        # get area status if available
        try :
            for strong_tag in cg_soup.find_all('strong'):
                if ('Area Status' in unidecode(strong_tag.text)):
                    status = unidecode(strong_tag.next_sibling).strip()
        except Exception:
            print('couldnt get area status')


        print("getting location")
        # get lat, long, altitude
        try :
            lat = cg_soup.find_all('div', text=re.compile('Latitude'))
            div = [row.next_sibling.next_sibling for row in lat]
            latitude  = div[0].text.strip()

            lng = cg_soup.find_all('div', text=re.compile('Longitude'))
            div = [row.next_sibling.next_sibling for row in lng]
            longitude  = div[0].text.strip()

            el = cg_soup.find_all('div', text=re.compile('Elevation'))
            div = [row.next_sibling.next_sibling for row in el]
            elevation  = div[0].text.strip()

            # get site usage, type, num sites, site info

        except Exception:
            print('couldnt get location info')

        # table[0] is the basic info table

        try :
            tables = cg_soup.find_all('div', {'class': 'tablecolor'})
        except Exception:
            print('couldnt get tables')

        try :
            print(len(tables))
            rows = tables[0].find_all('tr')
            for row in rows:
                
                if row.th.text == 'Reservations:':
                    reservations = unidecode(row.td.text).strip()
                if row.th.text == 'Open Season:':
                    openseason = unidecode(row.td.text).strip()
                if row.th.text == 'Current Conditions:':
                    print(unidecode(row.td.text))
                    conditions = unidecode(row.td.text).strip()
                if row.th.text == 'Water:':
                    water = unidecode(row.td.text).strip()
                if row.th.text == 'Restroom:':
                    restroom = unidecode(row.td.text).strip()
        except Exception as ex:
            print('couldnt get basic campground info')
            print(ex)

        # table 1 is the campground info
        try:
            rows = tables[1].find_all('tr')

            for row in rows:
                if row.td.text == 'Reservation Info':
                    reserveinfo = unidecode(row.td.next_sibling.text).strip()
                if row.td.text == 'No. of Sites':
                    numsites = unidecode(row.td.next_sibling.text).strip()
        except Exception:
            print('couldnt get campsite availability info')
        
        # assemble into DataFrame
        df_cg = df_cg.append(pd.DataFrame ({
            'latitude': [latitude],
            'longitude': [longitude],
            'elevation': [elevation],
            'facilityname' : [campground['facilityname']],
            'facilityurl' : [site_url],
            'status' : [status],
            'water' : [water],
            'restroom' : [restroom],
            'reservations': [reservations],
            'conditions': [conditions],
            'numsites': [numsites]
            }))
    return df_cg